In [24]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [32]:
data = pd.read_csv("data/final_data.csv")
data.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'data/final_data.csv'

In [15]:
data.info()

NameError: name 'data' is not defined

In [ ]:
data.loc[:, 'lot_size'] = data['info'].str.extract(r'Lot size: ([0-9]+)').values.reshape(-1)
data.head(5)

#Посмотреть корреляцию

In [ ]:
plt.figure(figsize = (8,6))
sns.heatmap(data.corr())
plt.show()

#Удаляем ненужные столбцы

In [ ]:
drop_columns = ['Unnamed: 0', 'info', 'address', 'latitude', 'longitude', 'zpid']
data.drop(drop_columns, axis=1, inplace = True)

#вытаскиваем год из даты

In [ ]:
data.loc[:, 'sold_year'] = pd.to_datetime(data['lastsolddate']).dt.year
data.drop(['lastsolddate'], axis=1, inplace=True)
data.head(5)

#РАБОТА С ПРОПУСКАМИ И ПЕРЕМЕННЫМИ

In [ ]:
data_copy = data.copy()
data_copy.head(5)

In [ ]:
data.describe()

In [ ]:
index_for_drop = data_copy.sample(1000).index


In [ ]:
data_copy.loc[index_for_drop, 'bathrooms'] = np.nan
data_copy.loc[index_for_drop, 'bathrooms']

##заполнение пропусков

##нулями

In [ ]:
data_copy['bathrooms'].fillna(0.0)

##медианами

In [ ]:
fill_value = data_copy['bathrooms'].median()
data_copy['bathrooms'].fillna(fill_value)

##средними

In [ ]:
fill_value = data_copy['bathrooms'].mean()
data_copy['bathrooms'].fillna(fill_value)

In [ ]:
data_copy.loc[:, 'bathrooms_na'] = data_copy['bathrooms'].isna().astype(int)
data_copy['bathrooms'].fillna(0.0, inplace=True)
data_copy.head(5)

In [ ]:
data_copy.loc[index_for_drop, 'bathrooms']

In [ ]:
data.head(5)

In [ ]:
data['z_address'].value_counts()

In [ ]:
data['z_address'].str.extract(r'([A-Z0-9]{0,3}[A-Za-z]+\s?[A-Za-z\s]+)').values.reshape(-1)

In [ ]:
data.loc[:,'z_address'] = data['z_address'].str.extract(r'([A-Z0-9]{0,3}[A-Za-z]+\s?[A-Za-z\s]+)').values.reshape(-1)
data['z_address'].value_counts()

In [ ]:
data.loc[data['bathrooms'] % 1 != 0.0, 'bathrooms']

In [ ]:
a = np.floor(data['bathrooms'].values)

In [ ]:
data.loc[:, 'full_bathrooms'] = a
data.loc[:, 'add_bathrooms'] = data['bathrooms'] - data['full_bathrooms']
data.drop(['bathrooms'], axis=1, inplace=True)
data.head(5)

In [ ]:
data['add_bathrooms'].value_counts()

In [ ]:
data.info()

In [ ]:
data['lot_size']

In [ ]:
zind_cnts = data['zindexvalue'].value_counts()
data.loc[data['zindexvalue'].isin(zind_cnts[zind_cnts <= 50].index), 'zindexvalue'] = 'other'

In [ ]:
data['zindexvalue'].value_counts()

In [ ]:
usecode_cnts = data['usecode'].value_counts()
data.loc[data['usecode'].isin(usecode_cnts[usecode_cnts <= 50].index), 'usecode'] = 'Miscellaneous'

In [ ]:
data['usecode'].value_counts()

### пример кодирования

In [ ]:
ohe_encoder = OneHotEncoder()
ohe_usecode = ohe_encoder.fit_transform(data[['usecode']]).toarray()
ohe_usecode = pd.DataFrame(ohe_usecode, columns=ohe_encoder.categories_[0], index=data.index)
ohe_usecode.head(5)

In [ ]:
label_encoder = LabelEncoder()
label_usecode = label_encoder.fit_transform(data[['usecode']])
label_usecode = pd.DataFrame(label_usecode, columns=['usecode_label'], index=data.index)
label_usecode.head(5)

In [ ]:
data = data.join(ohe_usecode)
data.drop(['usecode'], axis=1, inplace=True)
data.head(5)